In [68]:
import os
import sys
import copy
from pathlib import Path

import scipy.io
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

In [90]:
root = Path(os.getcwd()).parent
sys.path.append(root)

dataset_name = ['mpii']
dataset_path = list(map(lambda x: os.path.join(root, 'data', x), dataset_name))

# Format of filenames = [[mpii_img_1, mpii_img_2, ... (mpii_img_k)]]
filenames_ = list(map(lambda path, name: open(os.path.join(path, '{}_filenames.txt'.format(name))), dataset_path, dataset_name))
filenames = list(map(lambda f: f.read().split(), filenames_))
_ = list(map(lambda f: f.close(), filenames_))

In [70]:
count = 0
# index to joint name conversion
mpii_idx_to_jnt = {0: 'rankl', 1: 'rknee', 2: 'rhip', 5: 'lankl', 4: 'lknee', 3: 'lhip',
                   6: 'pelvis', 7: 'thorax', 8: 'upper_neck', 11: 'relb', 10: 'rwri', 9: 'head',
                   12: 'rsho', 13: 'lsho', 14: 'lelb', 15: 'lwri'}

# This template will then be updated as and when we read ground truth
mpii_template = dict([(mpii_idx_to_jnt[i], []) for i in range(16)])

# Load the mat file.
matlab_mpii = scipy.io.loadmat(os.path.join(dataset_path[0], 'joints.mat'), struct_as_record=False)['RELEASE'][0, 0]
num_images = annotation_mpii = matlab_mpii.__dict__['annolist'][0].shape[0]

In [91]:
matlab_mpii.__dict__.keys()

dict_keys(['_fieldnames', 'annolist', 'img_train', 'version', 'single_person', 'act', 'video_list'])

In [93]:
len([item[0] for item in matlab_mpii.__dict__['video_list'][0] if not isinstance(item[0], list)])


2821

In [95]:
print(set([item[0] for item in matlab_mpii.__dict__['video_list'][0] if not isinstance(item[0], list)]))

{'jxJceIuqo_I', '2sVh5Zy3JUg', 'htiCL7avJKo', 'ngOALrNtE-Q', 'yETy-dzmr8Y', '3hEVbrnufgQ', '3J1DOHC24m4', 'TdkSLL_xcAY', 'snyyXD0RAy0', 'AuaC8GdQkSI', 'HAitIFTfAuo', 'hUU0O6czirA', 'Dp11usjNR4Q', 'bJGxM24PRX4', 'aL5nJXePlsA', 'h0D7FLOKU10', 'jrxDwT2arso', 'I8EvKTt_rbI', 'gWg4vqCdoYg', 'Hor3GYvgKQE', 'y6HxORx0g-g', 'IdNGJo4TJmU', 'd_yKAXIwNpk', 'rIBRcQdzWQs', 'WxwlJQryMgI', 'KB8HyzvPTVg', 'avF-rCAce0Q', 'dk8ZESPyY-E', 'CQYJd-7iQb8', '-vY2uUmv6Dk', 'p5rPdrzB1YY', 'FshoSr1qF9w', 't7lad_RXoQo', 'x7bRnYvPZ0Y', 'wg0DEP_Dv4o', 'cyJHA_jrzaI', 'cIZHWPSNKyI', 'ITS0UoPQzq0', 'KfavaCFIm8M', 'LC2DDdhhkgY', '6k9PyP1yKiE', 'xNy2b-OXMZg', 'URe38Qi0oA8', 'A5WoMB9cd-U', '431CnRJAUUQ', 'L69u7csJ_oU', 'n5CbwmGbnOs', 'bMxwbCAIhOw', 'lPwM2vn3bT0', 'Dz7s8Uy4q0I', 'mFgGUMzT0CM', '-9O0D5kO0Tk', 'iKT2BZysEGY', 'BDcuMiglyyg', 'tjM6-wAi9f8', 'Zw8iljkD1So', 'hSRcyh0XLao', 'mXkhIq9jNIU', 'L1aGPeqWNFw', 'CqnxSlyvr-k', 'GKJ1IKqWtfg', 'luBSMGFVjls', 'MT852hP9wVk', '-8JbTKR50rk', 'ObUpaW_5L_E', '4niMx5vtucU', 'bbSOLlso

In [97]:
(matlab_mpii.__dict__['annolist'][0]).shape

(24987,)

In [24]:
count = 0
# index to joint name conversion
mpii_idx_to_jnt = {0: 'rankl', 1: 'rknee', 2: 'rhip', 5: 'lankl', 4: 'lknee', 3: 'lhip',
                   6: 'pelvis', 7: 'thorax', 8: 'upper_neck', 11: 'relb', 10: 'rwri', 9: 'head',
                   12: 'rsho', 13: 'lsho', 14: 'lelb', 15: 'lwri'}

# This template will then be updated as and when we read ground truth
mpii_template = dict([(mpii_idx_to_jnt[i], []) for i in range(16)])

# Load the mat file.
matlab_mpii = scipy.io.loadmat(os.path.join(dataset_path[0], 'joints.mat'), struct_as_record=False)['RELEASE'][0, 0]
num_images = annotation_mpii = matlab_mpii.__dict__['annolist'][0].shape[0]

# Load images and GT in batches of 200
initial_index = 0
batch = 200
while initial_index < num_images:
    # Initialize empty placeholder
    img_dict = {'mpii': {'img': [], 'img_name': [], 'img_pred': [], 'img_gt': []}}
    
    # Iterate over each image
    for img_idx in tqdm(range(initial_index, min(initial_index + batch, num_images))):
        annotation_mpii = matlab_mpii.__dict__['annolist'][0, img_idx]
        train_test_mpii = matlab_mpii.__dict__['img_train'][0, img_idx].flatten()[0]
        person_id = matlab_mpii.__dict__['single_person'][img_idx][0].flatten()

        # Load the individual image. Throw an exception if image corresponding to filename not available.
        img_name = annotation_mpii.__dict__['image'][0, 0].__dict__['name'][0]
        try:
            image = plt.imread(os.path.join(dataset_path[0], 'images', img_name))
        except FileNotFoundError:
            #print('Could not load filename: {}'.format(img_name))
            continue

        # Avoid modifying the template and create a copy
        gt_per_image = copy.deepcopy(mpii_template)

        # Flag is set to true if atleast one person exists in the image with joint annotations.
        # If Flag == True, then the image and GT is considered for visualization, else skip
        annotated_person_flag = False
        
        # Iterate over persons
        for person in (person_id - 1):
            try:
                annopoints_img_mpii = annotation_mpii.__dict__['annorect'][0, person].__dict__['annopoints'][0, 0]
                num_joints = annopoints_img_mpii.__dict__['point'][0].shape[0]

                # Iterate over present joints
                for i in range(num_joints):
                    x = annopoints_img_mpii.__dict__['point'][0, i].__dict__['x'].flatten()[0]
                    y = annopoints_img_mpii.__dict__['point'][0, i].__dict__['y'].flatten()[0]
                    id_ = annopoints_img_mpii.__dict__['point'][0, i].__dict__['id'][0][0]
                    vis = annopoints_img_mpii.__dict__['point'][0, i].__dict__['is_visible'].flatten()
                    print(x, y, id_, vis)
                    # No entry corresponding to visible
                    if vis.size == 0:
                        vis = 1
                    else:
                        vis = vis.item()

                    gt_per_joint = np.array([x, y, vis]).astype(np.float16)
                    gt_per_image[mpii_idx_to_jnt[id_]].append(gt_per_joint)

                annotated_person_flag = True
            except KeyError:
                count += 1
                # Person 'x' could not have annotated joints, hence move to person 'y'
                continue

        if not annotated_person_flag:
            count += 1
            continue

        # Update the template copy with image, name and ground truth
        img_dict['mpii']['img'].append(image)
        img_dict['mpii']['img_name'].append(img_name)
        img_dict['mpii']['img_gt'].append(gt_per_image)
    
    initial_index += batch

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\chhag\Documents\DSE\Masterarbeit\advanced_landmark_detection\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\chhag\AppData\Local\Temp\ipykernel_17604\1503505785.py", line 11, in <module>
    matlab_mpii = scipy.io.loadmat(os.path.join(dataset_path[0], 'joints.mat'), struct_as_record=False)['RELEASE'][0, 0]
  File "c:\Users\chhag\Documents\DSE\Masterarbeit\advanced_landmark_detection\venv\lib\site-packages\scipy\io\matlab\_mio.py", line 227, in loadmat
    matfile_dict = MR.get_variables(variable_names)
  File "c:\Users\chhag\Documents\DSE\Masterarbeit\advanced_landmark_detection\venv\lib\site-packages\scipy\io\matlab\_mio5.py", line 330, in get_variables
    res = self.read_var_array(hdr, process)
  File "c:\Users\chhag\Documents\DSE\Masterarbeit\advanced_landmark_detection\venv\lib\site-packages\scipy\io\matlab\_mio5.py", line 290,

In [15]:
img_dict

{'mpii': {'img': [], 'img_name': [], 'img_pred': [], 'img_gt': []}}